In [5]:
import pandas as pd

In [2]:
######멍충 정우가 작성한 부분#######
#크롤링(selenium/chromedriver -> 작품페이지)
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re

#df 생성
import pandas as pd
import numpy as np


#call chromedriver
driver = Chrome('./chromedriver')
url = 'https://comic.naver.com/webtoon/creation'
driver.get(url)

#전체 내용 가져오기
cartoon_list = driver.find_elements(By.CLASS_NAME,"section")
#video_list = driver.find_elements(By.CLASS_NAME,"style-scope ytd-video-renderer")

#작품별 페이지에서 뽑을 수 있는 정보들 

title = [] #웹툰 제목 
webtoon_link = [] #웹툰 링크
status = [] #연재 상태(완결:1 연재중:0)
thumb_link = [] #웹툰 썸네일 이미지 링크

for cartoon_start in cartoon_list:
    now_li = cartoon_start.find_elements(By.TAG_NAME,"li")
    print(now_li)
    for li in now_li:
        a = li.find_element(By.TAG_NAME,'a')
        title.append(a.get_attribute("title"))
        webtoon_link.append(a.get_attribute("href"))
        img = a.find_element(By.TAG_NAME,"img")
        thumb_link.append(img.get_attribute('src'))
        
        try:
            state = a.find_element(By.CLASS_NAME,"finish")
            if state:
                status.append(1)
        except:
            status.append(0)
            
#webtoon_link 안에 들어가서 뽑을 수 있는 내용들 -> 장르, 몇세이용가, 작가명  (try: 시리즈 있는지 없는지 if 있다면 : 시리즈 링크 가져오기 ,없다면 시작일 최신화 날짜,)
#크롤링(상세페이지)
import requests
from bs4 import BeautifulSoup
import time
auth = [] #작가
likes = [] #좋아요 개수 
genres = [] #장르
ages = [] #이용 가능 나이
intro = [] #소개글
series_url = [] #시리즈 url 
r_rated = [] #19세 여부
episodes = [None]*len(webtoon_link) #현재까지 올라온 에피소드 갯수(유료 포함)
first_register_date = [] #웹툰 첫번째 등록날짜
last_register_date = [] #웹툰 마지막 등록 날짜

#call_web_df = pd.read_csv("./webtoon.csv",index_col=0)
#len(call_web_df)
for x in range(len(webtoon_link)):
    #except likes
    print(x)
    req = requests.get(webtoon_link[i])

    html = req.text
    
    soup = BeautifulSoup(html,'html.parser')
    
    author = soup.select('#content > div.comicinfo > div.detail > h2 > span.wrt_nm') #작가 정보
    gen = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre') #장르 
    age = soup.select("#content > div.comicinfo > div.detail > p.detail_info > span.age") #나이
    intr = soup.select("#content > div.comicinfo > div.detail > p:nth-child(2)") #소개글
    series = soup.select(".band_banner.v2 > td > a") #시리즈 주소

    # 태형
    register_list = soup.find("table","viewList").find_all("td","num")
    first_register_date.append(register_list[-1].text)  
    last_register_date.append(register_list[0].text)

    if soup.find("div",class_="comicinfo").find('span',class_='mark_adult_thumb') != None: #19세 여부
        r_rated.append(1)
    else:
        r_rated.append(0)
    
    for i in author:
        auth.append(i.text.strip())
    
    for i in gen:
        genres.append(i.text)
    
    if age==[]: #만약 비어있으면 전체 연령가로 바꿔버리기
        ages.append("전체연령가")
    else:
        for i in age:
            ages.append(i.text)
    
    for i in intr:
        intro.append(i.text)
    

    if series==[]: #만약 비어있으면 그냥 none 으로 주고 나중에 selenium 으로 뽑아내기 
        series_url.append(None)
        episodes[x] = soup.find("table",class_="viewList").find('td',class_='title').find('a')['onclick'].split("'")[5] #시리즈가 없는 웹툰의 최종화, 시리즈가 있는건 selenium으로
    else:
        for i in series:
            series_url.append(i.get('href'))


/var/folders/b2/xf80p57564z4z_vr5vzw0nd80000gn/T/ipykernel_29514/4110736593.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('./chromedriver')


[<selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="d934cb47-31ce-4ead-bc7f-19725f869d24")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="b5748866-ebbc-44c8-9279-0004de3dfc05")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="b2dfac37-8463-4141-9ce6-9e90e66c4954")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="292d7013-191a-4164-aaae-91f030e3992c")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="c7bcd036-d6db-4e0e-ada2-9e538bce36f6")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="30b94ada-a91d-4985-8ef7-fb9d64cc6e03")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="3c4a5c89-e00f-4483-a39c-37c24116

[<selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="d65ff284-e761-4704-9cf9-20a7d38ff331")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="4275e9c1-21be-43c8-97c5-c6eef2ea0254")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="af15fa99-89fd-4145-a482-6613fd52cdd7")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="eada2129-b82e-4240-aa3b-322907c924c4")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="ebea29bb-33de-4b47-a161-1311713e9aff")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="a349f06b-635c-4049-9fc8-08880a474e04")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="d1dbafc4-90bb-489b-9339-6ee47a67

[<selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="4d5f8b42-236c-4a72-85d5-9bcc3819d06b")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="7c772a1b-2057-424d-a06f-e98c307984d5")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="076bd6c7-8854-4e81-b645-bb6ba1803bb9")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="643f1a6d-f4c6-4082-a8f7-e87fcff76667")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="caccebc2-2885-402c-a735-98b28173ac65")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="4db512a3-785d-4f35-b249-20ce775e49fc")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="08e0c5f7-7d69-4192-a2a8-9d417d8c

[<selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="030e3d8d-8596-40e4-b011-a64614c306c7")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="ac9a7000-5b6f-4cd8-9d53-e14894f53b8e")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="2c6de8c9-77d3-461d-9c9d-b2a91fb049cf")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="076edad9-ab96-44d5-b2d7-3614daa75641")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="5f811929-8beb-4dca-a223-d00b21a53bb0")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="e7d32833-d6ea-4cc3-aa22-c0a189d65258")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="bad2fb71-cebf-4197-b120-b0cbb867

...

[<selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="d87d7d8f-08cd-4e5c-9e4a-4ef0d95d0720")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="df5d357d-2ec8-495b-8938-8d4bbf29b0b5")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="db77f7d3-d526-414c-97a3-d36f5a83037a")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="1155349b-d9da-44ef-abfe-baf5217192c9")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="76dfba92-dd1f-4fb8-927d-faa225d4f574")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="ad490801-9c02-4dc7-841d-c87daa10205e")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="9673058b-6ce3-41b6-8c25-4fa49392

[<selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="1bdaa2cf-3d44-4c22-ac36-057e5e11cbfd")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="f20bce4e-a505-4bb0-928d-ec5f5a2431df")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="eaa63972-3b96-4ba4-90d2-1c4fa5867f83")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="9881b71a-f9b2-48e2-8fe9-7b22f9936a34")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="bb75e4a5-a137-482f-8eac-4aff0362fb69")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="c2a6bfb2-a1d5-4245-81b0-b9bf5a27bb69")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="9191d568-3b52-482c-bc64-e156ae00

[<selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="a73906ea-24db-4601-84bb-fd57530f0240")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="9e0f12c8-4644-4117-9343-55c7f6759e61")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="11deb30f-f772-4038-b93d-55f87b4a0be6")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="db72519b-a29d-4a1c-940a-227f1293f2b2")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="be6b6a3d-cf7c-4e9e-a979-c3b044a5f12a")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="dd77e3ad-4fec-45e0-8deb-30896060636c")>, <selenium.webdriver.remote.webelement.WebElement (session="8d8142c7696450b19f298d733c56c4eb", element="84ed4c07-053d-4e6a-933c-79996445

0


NameError: name 'i' is not defined

In [ ]:
#잘 뽑혔는지 확인하기
print(len(title))
print(len(webtoon_link))
print(len(thumb_link))
print(len(status))
print(len(auth))
print(len(likes))
print(len(genres))
print(len(ages))
print(len(intro))
print(len(series_url))
print(len(r_rated))
print(len(episodes))
print(len(first_register_date))
print(len(last_register_date))


In [6]:
#webtoon_link 안에 들어가서 뽑을 수 있는 내용들 -> 장르, 몇세이용가, 작가명  (try: 시리즈 있는지 없는지 if 있다면 : 시리즈 링크 가져오기 ,없다면 시작일 최신화 날짜,)
#크롤링(상세페이지)
import requests
from bs4 import BeautifulSoup
import time
auth = [] #작가
likes = [] #좋아요 개수 
genres = [] #장르
ages = [] #이용 가능 나이
intro = [] #소개글
series_url = [] #시리즈 url 
r_rated = [] #19세 여부
episodes = [None]*len(webtoon_link) #현재까지 올라온 에피소드 갯수(유료 포함)
first_register_date = [] #웹툰 첫번째 등록날짜
last_register_date = [] #웹툰 마지막 등록 날짜

#call_web_df = pd.read_csv("./webtoon.csv",index_col=0)
#len(call_web_df)
for x in range(len(webtoon_link)):
    #except likes
    print(x)
    req = requests.get(webtoon_link[x])

    html = req.text
    
    soup = BeautifulSoup(html,'html.parser')
    
    author = soup.select('#content > div.comicinfo > div.detail > h2 > span.wrt_nm') #작가 정보
    gen = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre') #장르 
    age = soup.select("#content > div.comicinfo > div.detail > p.detail_info > span.age") #나이
    intr = soup.select("#content > div.comicinfo > div.detail > p:nth-child(2)") #소개글
    series = soup.select(".band_banner.v2 > td > a") #시리즈 주소

    # 태형
    register_list = soup.find("table","viewList").find_all("td","num")
    first_register_date.append(register_list[-1].text)  
    last_register_date.append(register_list[0].text)

    if soup.find("div",class_="comicinfo").find('span',class_='mark_adult_thumb') != None: #19세 여부
        r_rated.append(1)
    else:
        r_rated.append(0)
    
    for i in author:
        auth.append(i.text.strip())
    
    for i in gen:
        genres.append(i.text)
    
    if age==[]: #만약 비어있으면 전체 연령가로 바꿔버리기
        ages.append("전체연령가")
    else:
        for i in age:
            ages.append(i.text)
    
    for i in intr:
        intro.append(i.text)
    

    if series==[]: #만약 비어있으면 그냥 none 으로 주고 나중에 selenium 으로 뽑아내기 
        series_url.append(None)
        episodes[x] = soup.find("table",class_="viewList").find('td',class_='title').find('a')['onclick'].split("'")[5] #시리즈가 없는 웹툰의 최종화, 시리즈가 있는건 selenium으로
    else:
        for i in series:
            series_url.append(i.get('href'))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043


In [7]:
#잘 뽑혔는지 확인하기
print(len(title))
print(len(webtoon_link))
print(len(thumb_link))
print(len(status))
print(len(auth))
print(len(likes))
print(len(genres))
print(len(ages))
print(len(intro))
print(len(series_url))
print(len(r_rated))
print(len(episodes))
print(len(first_register_date))
print(len(last_register_date))

#likes 빼고 다 값 들어옴 

2044
2044
2044
2044
2044
0
2044
2044
2044
2044
2044
2044
2044
2044


In [45]:
#성인 인증을 위해 우선 실행 -> 로그인 후 아래 소스코드 실행
driver = Chrome('./chromedriver')
driver.get(webtoon_link[1])

/var/folders/b2/xf80p57564z4z_vr5vzw0nd80000gn/T/ipykernel_29514/1337975447.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('./chromedriver')


In [25]:
likes = []

In [46]:
for x in range(len(likes),len(webtoon_link)):
    print(x)
    #call selenium for likes

    url = webtoon_link[x]
    driver.get(url)
    
    cartoon_page = driver.find_elements(By.TAG_NAME,"body")
    time.sleep(2)
    for i in cartoon_page:
        likes.append(i.find_element(By.CSS_SELECTOR,"#content > div.comicinfo > div.detail > ul > li:nth-child(5) > div > a > em").text)

    #시리즈가 있는 웹툰의 최종화
    
    if series_url[x] != None:
        driver.get(series_url[x])
        driver.get(driver.current_url.split('&')[0]+'&sortOrder=DESC#priceInfo')
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source)
        last_episode = soup.find('tbody', id='volumeList').find('tr')['class'][0].split('_')[3]
        episodes[x] = last_episode

1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576


In [28]:
print(len(likes))

35


In [47]:
#잘 뽑혔는지 확인하기
print(len(title))
print(len(webtoon_link))
print(len(thumb_link))
print(len(status))
print(len(auth))
print(len(likes))
print(len(genres))
print(len(ages))
print(len(intro))
print(len(series_url))
print(len(r_rated))
print(len(episodes))
print(len(first_register_date))
print(len(last_register_date))

2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044


In [ ]:
#성인 인증을 위해 우선 실행 -> 로그인 후 아래 소스코드 실행
driver = Chrome('./chromedriver')

In [54]:
print(series_url)

['https://series.naver.com/comic/detail.series?originalProductId=354928', 'https://series.naver.com/comic/detail.series?originalProductId=218870', 'https://series.naver.com/comic/detail.series?originalProductId=371133', 'https://series.naver.com/comic/detail.series?originalProductId=354284', None, None, 'https://series.naver.com/comic/detail.series?originalProductId=481577', 'https://series.naver.com/comic/detail.series?originalProductId=459863', 'https://series.naver.com/comic/detail.series?originalProductId=388684', 'https://series.naver.com/comic/detail.series?originalProductId=457822', 'https://series.naver.com/comic/detail.series?originalProductId=433200', 'https://series.naver.com/comic/detail.series?originalProductId=364808', 'https://series.naver.com/comic/detail.series?originalProductId=403326', 'https://series.naver.com/comic/detail.series?originalProductId=298204', 'https://series.naver.com/comic/detail.series?originalProductId=481008', 'https://series.naver.com/comic/detail

In [57]:
#그냥 별점 구할 수 있음
rates = []

for x in range(len(webtoon_link)):
    print(x)

    now = str(series_url[x])
    print(now)
    if now =='None':
        print("'",x,"'",": 0")
        rates.append(0)
    else:
        driver.get(now)
        time.sleep(1)
        ct = driver.find_elements(By.CSS_SELECTOR,"#content > div.end_head > div.score_area > em")
        
        for x in ct:
            rates.append(x.text)

0
https://series.naver.com/comic/detail.series?originalProductId=354928
1
https://series.naver.com/comic/detail.series?originalProductId=218870
2
https://series.naver.com/comic/detail.series?originalProductId=371133
3
https://series.naver.com/comic/detail.series?originalProductId=354284
4
None
' 4 ' : 0
5
None
' 5 ' : 0
6
https://series.naver.com/comic/detail.series?originalProductId=481577
7
https://series.naver.com/comic/detail.series?originalProductId=459863
8
https://series.naver.com/comic/detail.series?originalProductId=388684
9
https://series.naver.com/comic/detail.series?originalProductId=457822
10
https://series.naver.com/comic/detail.series?originalProductId=433200
11
https://series.naver.com/comic/detail.series?originalProductId=364808
12
https://series.naver.com/comic/detail.series?originalProductId=403326
13
https://series.naver.com/comic/detail.series?originalProductId=298204
14
https://series.naver.com/comic/detail.series?originalProductId=481008
15
https://series.naver.c

135
https://series.naver.com/comic/detail.series?originalProductId=432413
136
https://series.naver.com/comic/detail.series?originalProductId=469999
137
None
' 137 ' : 0
138
https://series.naver.com/comic/detail.series?originalProductId=477929
139
https://series.naver.com/comic/detail.series?originalProductId=472487
140
https://series.naver.com/comic/detail.series?originalProductId=488013
141
https://series.naver.com/comic/detail.series?originalProductId=240603
142
https://series.naver.com/comic/detail.series?originalProductId=331450
143
https://series.naver.com/comic/detail.series?originalProductId=400975
144
https://series.naver.com/comic/detail.series?originalProductId=208415
145
https://series.naver.com/comic/detail.series?originalProductId=450818
146
https://series.naver.com/comic/detail.series?originalProductId=379453
147
https://series.naver.com/comic/detail.series?originalProductId=356452
148
https://series.naver.com/comic/detail.series?originalProductId=372464
149
https://serie

259
https://series.naver.com/comic/detail.series?originalProductId=331200
260
https://series.naver.com/comic/detail.series?originalProductId=472676
261
None
' 261 ' : 0
262
None
' 262 ' : 0
263
https://series.naver.com/comic/detail.series?originalProductId=407561
264
https://series.naver.com/comic/detail.series?originalProductId=165240
265
https://series.naver.com/comic/detail.series?originalProductId=379926
266
https://series.naver.com/comic/detail.series?originalProductId=358584
267
https://series.naver.com/comic/detail.series?originalProductId=407156
268
None
' 268 ' : 0
269
https://series.naver.com/comic/detail.series?originalProductId=400978
270
https://series.naver.com/comic/detail.series?originalProductId=467945
271
https://series.naver.com/comic/detail.series?originalProductId=436046
272
https://series.naver.com/comic/detail.series?originalProductId=475688
273
https://series.naver.com/comic/detail.series?originalProductId=228264
274
None
' 274 ' : 0
275
https://series.naver.com

384
https://series.naver.com/comic/detail.series?originalProductId=369187
385
https://series.naver.com/comic/detail.series?originalProductId=482520
386
https://series.naver.com/comic/detail.series?originalProductId=350007
387
https://series.naver.com/comic/detail.series?originalProductId=395935
388
https://series.naver.com/comic/detail.series?originalProductId=404249
389
https://series.naver.com/comic/detail.series?originalProductId=442532
390
https://series.naver.com/comic/detail.series?originalProductId=443380
391
https://series.naver.com/comic/detail.series?originalProductId=436537
392
https://series.naver.com/comic/detail.series?originalProductId=483878
393
https://series.naver.com/comic/detail.series?originalProductId=383486
394
https://series.naver.com/comic/detail.series?originalProductId=399970
395
https://series.naver.com/comic/detail.series?originalProductId=396018
396
https://series.naver.com/comic/detail.series?originalProductId=263210
397
None
' 397 ' : 0
398
https://serie

512
https://series.naver.com/comic/detail.series?originalProductId=474751
513
https://series.naver.com/comic/detail.series?originalProductId=380806
514
https://series.naver.com/comic/detail.series?originalProductId=448747
515
https://series.naver.com/comic/detail.series?originalProductId=198587
516
https://series.naver.com/comic/detail.series?originalProductId=201791
517
https://series.naver.com/comic/detail.series?originalProductId=431297
518
https://series.naver.com/comic/detail.series?originalProductId=466286
519
https://series.naver.com/comic/detail.series?originalProductId=272919
520
https://series.naver.com/comic/detail.series?originalProductId=339778
521
https://series.naver.com/comic/detail.series?originalProductId=229230
522
None
' 522 ' : 0
523
None
' 523 ' : 0
524
https://series.naver.com/comic/detail.series?originalProductId=423883
525
https://series.naver.com/comic/detail.series?originalProductId=477629
526
https://series.naver.com/comic/detail.series?originalProductId=313

...

1390
https://series.naver.com/comic/detail.series?originalProductId=466320
1391
https://series.naver.com/comic/detail.series?originalProductId=481111
1392
https://series.naver.com/comic/detail.series?originalProductId=444540
1393
https://series.naver.com/comic/detail.series?originalProductId=216637
1394
https://series.naver.com/comic/detail.series?originalProductId=477644
1395
https://series.naver.com/comic/detail.series?originalProductId=439686
1396
https://series.naver.com/comic/detail.series?originalProductId=404095
1397
https://series.naver.com/comic/detail.series?originalProductId=483281
1398
https://series.naver.com/comic/detail.series?originalProductId=403122
1399
https://series.naver.com/comic/detail.series?originalProductId=362228
1400
https://series.naver.com/comic/detail.series?originalProductId=446404
1401
https://series.naver.com/comic/detail.series?originalProductId=474469
1402
https://series.naver.com/comic/detail.series?originalProductId=411743
1403
https://series.naver

1517
https://series.naver.com/comic/detail.series?originalProductId=428919
1518
https://series.naver.com/comic/detail.series?originalProductId=466135
1519
None
' 1519 ' : 0
1520
https://series.naver.com/comic/detail.series?originalProductId=467555
1521
https://series.naver.com/comic/detail.series?originalProductId=365738
1522
https://series.naver.com/comic/detail.series?originalProductId=394804
1523
https://series.naver.com/comic/detail.series?originalProductId=225403
1524
https://series.naver.com/comic/detail.series?originalProductId=195762
1525
None
' 1525 ' : 0
1526
https://series.naver.com/comic/detail.series?originalProductId=396046
1527
https://series.naver.com/comic/detail.series?originalProductId=455543
1528
None
' 1528 ' : 0
1529
https://series.naver.com/comic/detail.series?originalProductId=224959
1530
https://series.naver.com/comic/detail.series?originalProductId=430995
1531
https://series.naver.com/comic/detail.series?originalProductId=383712
1532
https://series.naver.com/c

1648
https://series.naver.com/comic/detail.series?originalProductId=477473
1649
https://series.naver.com/comic/detail.series?originalProductId=449359
1650
https://series.naver.com/comic/detail.series?originalProductId=236863
1651
https://series.naver.com/comic/detail.series?originalProductId=485448
1652
https://series.naver.com/comic/detail.series?originalProductId=109781
1653
https://series.naver.com/comic/detail.series?originalProductId=161938
1654
https://series.naver.com/comic/detail.series?originalProductId=225098
1655
https://series.naver.com/comic/detail.series?originalProductId=488629
1656
None
' 1656 ' : 0
1657
https://series.naver.com/comic/detail.series?originalProductId=328628
1658
https://series.naver.com/comic/detail.series?originalProductId=148997
1659
https://series.naver.com/comic/detail.series?originalProductId=148997
1660
https://series.naver.com/comic/detail.series?originalProductId=477666
1661
https://series.naver.com/comic/detail.series?originalProductId=430872
16

1781
https://series.naver.com/comic/detail.series?originalProductId=418919
1782
None
' 1782 ' : 0
1783
https://series.naver.com/comic/detail.series?originalProductId=275204
1784
https://series.naver.com/comic/detail.series?originalProductId=404250
1785
https://series.naver.com/comic/detail.series?originalProductId=367375
1786
https://series.naver.com/comic/detail.series?originalProductId=417566
1787
None
' 1787 ' : 0
1788
https://series.naver.com/comic/detail.series?originalProductId=313430
1789
https://series.naver.com/comic/detail.series?originalProductId=449531
1790
https://series.naver.com/comic/detail.series?originalProductId=395783
1791
https://series.naver.com/comic/detail.series?originalProductId=456560
1792
None
' 1792 ' : 0
1793
None
' 1793 ' : 0
1794
https://series.naver.com/comic/detail.series?originalProductId=111913
1795
https://series.naver.com/comic/detail.series?originalProductId=429738
1796
https://series.naver.com/comic/detail.series?originalProductId=486775
1797
htt

1910
https://series.naver.com/comic/detail.series?originalProductId=292399
1911
https://series.naver.com/comic/detail.series?originalProductId=362418
1912
https://series.naver.com/comic/detail.series?originalProductId=472080
1913
https://series.naver.com/comic/detail.series?originalProductId=473660
1914
https://series.naver.com/comic/detail.series?originalProductId=321385
1915
https://series.naver.com/comic/detail.series?originalProductId=466308
1916
https://series.naver.com/comic/detail.series?originalProductId=483251
1917
https://series.naver.com/comic/detail.series?originalProductId=431265
1918
https://series.naver.com/comic/detail.series?originalProductId=423690
1919
https://series.naver.com/comic/detail.series?originalProductId=466364
1920
https://series.naver.com/comic/detail.series?originalProductId=473561
1921
None
' 1921 ' : 0
1922
None
' 1922 ' : 0
1923
https://series.naver.com/comic/detail.series?originalProductId=485891
1924
https://series.naver.com/comic/detail.series?orig

In [58]:
#잘 뽑혔는지 확인하기
print(len(title))
print(len(webtoon_link))
print(len(thumb_link))
print(len(status))
print(len(auth))
print(len(likes))
print(len(genres))
print(len(ages))
print(len(intro))
print(len(series_url))
print(len(r_rated))
print(len(episodes))
print(len(first_register_date))
print(len(last_register_date))
print(len(rates))

2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044


In [81]:
backup_rates[4] =0
backup_rates[5] =0
backup_rates
print(backup_rates)

['9.6', '9.9', '9.9', '9.9', 0, 0, '9.4', '9.9', '9.5', '9.9', '9.1', '9.9', '10.0', '8.9', '9.1', '10.0', '9.3', '9.6', '9.4', '9.9', '9.9', '9.3', 0.0, '7.3', 9.969400000000006, '9.5', 9.914999999999997, '9.6', 9.79054545454546, '9.8', '9.6', '9.2', 9.975600000000007, '9.9', '9.9', '9.2', '9.7', '9.7', '9.1', '10.0', 9.977333333333332, '9.9', 9.486, '8.8', '9.8', 9.130999999999998, '9.8', '9.8', 9.932999999999998, '9.9', '9.9', '8.9', '9.3', '9.5', '10.0', '10.0', 9.888000000000002, '8.3', '8.8', '9.2', '10.0', '9.9', '9.6', '9.5', '9.8', '10.0', '9.8', '9.8', 9.949000000000005, '9.2', '9.7', '9.9', '9.9', 9.752, 9.355999999999998, 9.969, 9.962272727272737, 9.982153846153857, '8.7', 9.629000000000001, 9.811199999999998, '10.0', '9.9', '9.5', '9.4', '9.7', 9.927999999999999, '10.0', '9.9', '9.1', 9.983142857142866, 9.92097142857142, '9.9', 5.402299999999998, '9.9', '9.1', '9.4', 9.904, '9.5', '9.7', '9.4', '9.2', 9.754, '9.7', '10.0', '9.9', 0.0, '10.0', '9.9', '9.8', '9.8', '9.3', '9

In [80]:


rates

['9.6',
 '9.9',
 '9.9',
 '9.9',
 9.895999999999997,
 9.947846153846175,
 '9.4',
 '9.9',
 '9.5',
 '9.9',
 '9.1',
 '9.9',
 '10.0',
 '8.9',
 '9.1',
 '10.0',
 '9.3',
 '9.6',
 '9.4',
 '9.9',
 '9.9',
 '9.3',
 0.0,
 '7.3',
 9.969400000000006,
 '9.5',
 9.914999999999997,
 '9.6',
 9.79054545454546,
 '9.8',
 '9.6',
 '9.2',
 9.975600000000007,
 '9.9',
 '9.9',
 '9.2',
 '9.7',
 '9.7',
 '9.1',
 '10.0',
 9.977333333333332,
 '9.9',
 9.486,
 '8.8',
 '9.8',
 9.130999999999998,
 '9.8',
 '9.8',
 9.932999999999998,
 '9.9',
 '9.9',
 '8.9',
 '9.3',
 '9.5',
 '10.0',
 '10.0',
 9.888000000000002,
 '8.3',
 '8.8',
 '9.2',
 '10.0',
 '9.9',
 '9.6',
 '9.5',
 '9.8',
 '10.0',
 '9.8',
 '9.8',
 9.949000000000005,
 '9.2',
 '9.7',
 '9.9',
 '9.9',
 9.752,
 9.355999999999998,
 9.969,
 9.962272727272737,
 9.982153846153857,
 '8.7',
 9.629000000000001,
 9.811199999999998,
 '10.0',
 '9.9',
 '9.5',
 '9.4',
 '9.7',
 9.927999999999999,
 '10.0',
 '9.9',
 '9.1',
 9.983142857142866,
 9.92097142857142,
 '9.9',
 5.402299999999998,
 '9

In [113]:
#rates 에 내용 생성 
for i in range(len(rates)):
    if backup_rates[i] ==0: #시리즈 주소 없는 애들 
        print(i)
        count = 0 #여기에 합쳐줄거임 
        url = webtoon_link[i] #웹툰 주소 넣어주기 
        driver.get(url)
        cnt = 0
        try:
            while True:
                tbody = driver.find_elements(By.TAG_NAME,"tbody")
                next_btn = driver.find_element(By.CLASS_NAME,"next")
                
                for tr in tbody:
                    for td in tr.find_elements(By.TAG_NAME,"tr"):
                        cnt += 1
                        count += float(td.text.split('평점\n')[1][:4])
                        #print(cnt)
                next_btn.click()
        except:
            print("END")
            
        #zero division
        try:
            backup_rates[i] = count /cnt #최종 별점 추가하기 
        except:
            backup_rates[i] = 0.0
        print(backup_rates[i])

22


KeyboardInterrupt: 

In [122]:
cnt = 0


for i in range(len(rates)):
    if rates[i] == 0:
        print(i)
        print(title[i])
        
rates[22]=9.89
rates[106]=9.97
rates[274]=9.97
rates[345]=9.89
rates[403]=9.98
rates[441]=9.79
rates[454]=9.75
rates[523]=9.85
rates[532]=7.00
rates[552]=7.28
rates[649]=5.00
rates[695]=9.2
rates[754]=9.2
rates[787]=9.80
rates[999]=7.75
rates[1175]=9.98
rates[1206]=6.00
rates[1246]=9.97
rates[1267]=9.97
rates[1357]=9.97
rates[1589]=9.97
rates[1622]=9.00
rates[1674] = 9.95
rates[1690] = 9.63
rates[1757]=9.55
rates[1994]=9.55
rates[1996]=9.34
rates[1998]=9.80
rates[2001]=9.87

1674
타인지옥 - 연쇄살묘사건
1690
터치! 메리크리스마스
1757
폰령
1994
2011 루키 단편선
1996
2012 루키 단편선
1998
2013 루키 단편선
2001
2014 루키 단편선


In [123]:
for i in range(len(rates)):
    if rates[i] == 0:
        print(i)
        print(title[i])
        

In [124]:
#잘 뽑혔는지 확인하기
print(len(title))
print(len(webtoon_link))
print(len(thumb_link))
print(len(status))
print(len(auth))
print(len(likes))
print(len(genres))
print(len(ages))
print(len(intro))
print(len(series_url))
print(len(r_rated))
print(len(episodes))
print(len(first_register_date))
print(len(last_register_date))
print(len(rates))

2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044
2044


In [125]:
#df, csv 생성하기 (여기 위에 크롤링 소스 추가 하고 column 내용 덧붙여주세요)
column = ['title','webtoon_link','status','thumb_link','auth','genres','ages','intro','series_url','likes','r_rated','episodes','first_register_date','last_register_date','rates']

web_df = pd.DataFrame(zip(title,webtoon_link,status,thumb_link,auth,genres,ages,intro,series_url,likes,r_rated,episodes,first_register_date,last_register_date,rates))
web_df.columns = column

web_df.to_csv("./webtoon.csv")

In [129]:
call_csv = pd.read_csv("./webtoon.csv",index_col=0)
call_csv.head(5)

,title,webtoon_link,status,thumb_link,auth,genres,ages,intro,series_url,likes,r_rated,episodes,first_register_date,last_register_date,rates
0,가난을 등에 업은 소녀,https://comic.naver.com/webtoon/list?titleId=7...,1,https://shared-comic.pstatic.net/thumb/webtoon...,B급달궁 / 오은지,"스토리, 로맨스",전체연령가,흔하디 흔한 재벌후계자와 캔디도 울고 갈 박복한 가난소녀의 파란만장 동거기!인기작 ...,https://series.naver.com/comic/detail.series?o...,344,0,28.0,2020.10.19,2020.10.19,9.600
1,가담항설,https://comic.naver.com/webtoon/list?titleId=6...,1,https://shared-comic.pstatic.net/thumb/webtoon...,랑또,"스토리, 판타지",12세 이용가,이번 주인공은 돌이다!돌이지만 동료도 모으고 악당도 물리친다!랑또 작가표 동양 판타...,https://series.naver.com/comic/detail.series?o...,"99,999+",0,247.0,2016.06.01,2016.08.03,9.900
2,가령의 정체불명 이야기,https://comic.naver.com/webtoon/list?titleId=7...,1,https://shared-comic.pstatic.net/thumb/webtoon...,가령,"옴니버스, 드라마",15세 이용가,어느 날... 인어가 내게 말을 걸어왔다. 눈을 떠보니 총구가 있었다. 그리고 몸이...,https://series.naver.com/comic/detail.series?o...,"38,392",0,25.0,2019.08.19,2019.09.02,9.900
3,가비지타임,https://comic.naver.com/webtoon/list?titleId=7...,0,https://shared-comic.pstatic.net/thumb/webtoon...,2사장,"스토리, 스포츠",전체연령가,"한국 최초 한국형 고교스포츠 웹툰!열혈따윈 개나 줘, 낙오자들 뿐인 농구부의 운명은?",https://series.naver.com/comic/detail.series?o...,"81,510",0,149.0,2022.05.07,2022.11.19,9.900
4,가상&RPG,https://comic.naver.com/webtoon/list?titleId=7...,0,https://shared-comic.pstatic.net/thumb/webtoon...,주다현,"스토리, 판타지",12세 이용가,"""사람은 좀 죽여도 싸우는건 싫어해요."" 게임에서조차 집 바깥을 나가지 않는 '제리...",NaN,357,0,42.0,2022.08.17,2022.10.19,9.896


# 승환 

## 요일별 인기순

In [93]:
import requests
from bs4 import BeautifulSoup

week_list = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'dailyPlus']

# 요일별 인기순
popularity_order_list_by_day = []
# mon_popularity_order_by_day = []
# tue_popularity_order_by_day = []
# wed_popularity_order_by_day = []
# thu_popularity_order_by_day = []
# fri_popularity_order_by_day = []
# sat_popularity_order_by_day = []
# sun_popularity_order_by_day = []
# dailyPlus_popularity_order_by_day = []

for week_name in week_list:
    globals()['{}_popularity_order_by_day'.format(week_name)] = []
    print(globals()['{}_popularity_order_by_day'.format(week_name)])
    popularity_order_list_by_day.append(['{}_popularity_order_by_day'.format(week_name)])
    
print(popularity_order_list_by_day[1])
print(type(popularity_order_list_by_day[1]))


for k in range(0, len(week_list)):
    print('-------------------' + week_list[k] + '-------------------')
    url = 'https://m.comic.naver.com/webtoon/weekday?week=' + week_list[k]
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else : 
        print(response.status_code)

    # <span class="title_text">참교육</span>

    webtoon_title = soup.find_all("span", {"class":"title_text"})
    print(len(webtoon_title))
    for i in range(0, len(webtoon_title)):
        print(webtoon_title[i].text)
        popularity_order_list_by_day[k].append(webtoon_title[i].text)

[]
[]
[]
[]
[]
[]
[]
[]
['tue_popularity_order_by_day']
<class 'list'>
-------------------mon-------------------
81
참교육
버림받은 왕녀의 은밀한 침실
팔이피플
뷰티풀 군바리
퀘스트지상주의
장씨세가 호위무사
윈드브레이커
호랑신랑뎐
물어보는 사이
앵무살수
똑 닮은 딸
신화급 귀속 아이템을 손에 넣었다
북부 공작님을 유혹하겠습니다
이별 후 사내 결혼
순정말고 순종
오빠집이 비어서
퍼니게임
황제와의 하룻밤
또다시, 계약 부부
소녀의 세계
꼬리잡기
절대검감
잔불의 기사
리턴 투 플레이어
결혼생활 그림일기
다시쓰는 연애사
이제야 연애
사이다걸
루루라라 우리네 인생
아, 쫌 참으세요 영주님!
원작은 완결난 지 한참 됐습니다만
제왕: 빛과 그림자
인섹터
퇴근 후에 만나요
입술이 예쁜 남자
랭커
아슈타르테
미니어처 생활백서
더블클릭
굿바이 유교보이
집사, 주세요!
헬크래프트
오늘의 비너스
수영만화일기
히어로메이커
우산 없는 애
파운더
악녀 18세 공략기
그림자 신부
불청객
나만의 고막남친
루크 비셸 따라잡기
도깨비 부른다
메리의 불타는 행복회로
지옥연애환담
레지나레나 - 용서받지 못한 그대에게
하루의 하루
모노마니아
꿈의 기업
백호랑
세번째 로망스
흔들리는 세계로부터
경비실에서 안내방송 드립니다
모스크바의 여명
버그이터
최후의 금빛아이
행운을 부탁해!
역주행!
너를 돌려차는 방법
결혼공략
파견체
슈퍼스타 천대리
매지컬 급식:암살법사
사막에 핀 달
달로 만든 아이
아마도
신군
오로지 오로라
별을 쫓는 소년들
사랑의 헌옷수거함
디나운스
-------------------tue-------------------
82
1을 줄게
여신강림
김부장
하루만 네가 되고 싶어
마루는 강쥐
대학원 탈출일지
내가 키운 S급들
놓지마 정신줄 시즌3
중증외상센터 : 골든 아워
멸망 이후의 세계
시한부인 줄 알았어요!
하북팽가 막내아들
올가미
천마는 평범하게 살 수 없다
로잘린 보가트
집이 없어
나타나주세요!

## 요일별 여성_인기순

In [94]:
import requests
from bs4 import BeautifulSoup

week_list = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'dailyPlus']

# 요일별 여성_인기순
female_reader_popularity_order_list_by_day = []
# mon_female_reader_popularity_order_list_by_day = []
# ...

for week_name in week_list:
    globals()['{}_female_reader_popularity_order_list_by_day'.format(week_name)] = []
    print(globals()['{}_female_reader_popularity_order_list_by_day'.format(week_name)])
    female_reader_popularity_order_list_by_day.append(['{}_female_reader_popularity_order_list_by_day'.format(week_name)])
    
print(female_reader_popularity_order_list_by_day[1])
print(type(female_reader_popularity_order_list_by_day[1]))


for k in range(0, len(week_list)):
    print('-------------------' + week_list[k] + '-------------------')
    url = 'https://m.comic.naver.com/webtoon/weekday?sort=FEMALE_READER&week=' + week_list[k]
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else : 
        print(response.status_code)

    # <span class="title_text">참교육</span>

    webtoon_title = soup.find_all("span", {"class":"title_text"})
    print(len(webtoon_title))
    for i in range(0, len(webtoon_title)):
        print(webtoon_title[i].text)
        female_reader_popularity_order_list_by_day[k].append(webtoon_title[i].text)

[]
[]
[]
[]
[]
[]
[]
[]
['tue_female_reader_popularity_order_list_by_day']
<class 'list'>
-------------------mon-------------------
81
버림받은 왕녀의 은밀한 침실
팔이피플
물어보는 사이
호랑신랑뎐
똑 닮은 딸
북부 공작님을 유혹하겠습니다
이별 후 사내 결혼
순정말고 순종
오빠집이 비어서
참교육
황제와의 하룻밤
또다시, 계약 부부
뷰티풀 군바리
소녀의 세계
다시쓰는 연애사
이제야 연애
꼬리잡기
결혼생활 그림일기
루루라라 우리네 인생
사이다걸
원작은 완결난 지 한참 됐습니다만
아, 쫌 참으세요 영주님!
퇴근 후에 만나요
입술이 예쁜 남자
아슈타르테
퀘스트지상주의
굿바이 유교보이
미니어처 생활백서
오늘의 비너스
집사, 주세요!
퍼니게임
윈드브레이커
우산 없는 애
수영만화일기
그림자 신부
악녀 18세 공략기
나만의 고막남친
루크 비셸 따라잡기
불청객
장씨세가 호위무사
메리의 불타는 행복회로
모노마니아
지옥연애환담
레지나레나 - 용서받지 못한 그대에게
세번째 로망스
흔들리는 세계로부터
하루의 하루
신화급 귀속 아이템을 손에 넣었다
행운을 부탁해!
잔불의 기사
백호랑
모스크바의 여명
결혼공략
역주행!
너를 돌려차는 방법
인섹터
사막에 핀 달
앵무살수
달로 만든 아이
도깨비 부른다
경비실에서 안내방송 드립니다
리턴 투 플레이어
슈퍼스타 천대리
꿈의 기업
사랑의 헌옷수거함
오로지 오로라
별을 쫓는 소년들
랭커
최후의 금빛아이
더블클릭
절대검감
매지컬 급식:암살법사
헬크래프트
디나운스
제왕: 빛과 그림자
파견체
아마도
파운더
히어로메이커
버그이터
신군
-------------------tue-------------------
82
1을 줄게
여신강림
하루만 네가 되고 싶어
마루는 강쥐
대학원 탈출일지
시한부인 줄 알았어요!
놓지마 정신줄 시즌3
올가미
로잘린 보가트
중증외상센터 : 골든 아워
견우와 선녀
집이 없어
나타나주세요!
또 다른 사랑
플레이, 플리
내가 키운 S

## 요일별 남성_인기순

In [95]:
import requests
from bs4 import BeautifulSoup

week_list = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'dailyPlus']

# 요일별 남성_인기순
male_reader_popularity_order_list_by_day = []
# mon_male_reader_popularity_order_list_by_day = []
# ...

for week_name in week_list:
    globals()['{}_male_reader_popularity_order_list_by_day'.format(week_name)] = []
    print(globals()['{}_male_reader_popularity_order_list_by_day'.format(week_name)])
    male_reader_popularity_order_list_by_day.append(['{}_male_reader_popularity_order_list_by_day'.format(week_name)])
    
print(male_reader_popularity_order_list_by_day[1])
print(type(male_reader_popularity_order_list_by_day[1]))


for k in range(0, len(week_list)):
    print('-------------------' + week_list[k] + '-------------------')
    url = 'https://m.comic.naver.com/webtoon/weekday?sort=MALE_READER&week=' + week_list[k]
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else : 
        print(response.status_code)

    # <span class="title_text">참교육</span>

    webtoon_title = soup.find_all("span", {"class":"title_text"})
    print(len(webtoon_title))
    for i in range(0, len(webtoon_title)):
        print(webtoon_title[i].text)
        male_reader_popularity_order_list_by_day[k].append(webtoon_title[i].text)

[]
[]
[]
[]
[]
[]
[]
[]
['tue_male_reader_popularity_order_list_by_day']
<class 'list'>
-------------------mon-------------------
81
참교육
퀘스트지상주의
뷰티풀 군바리
장씨세가 호위무사
윈드브레이커
앵무살수
신화급 귀속 아이템을 손에 넣었다
퍼니게임
절대검감
리턴 투 플레이어
잔불의 기사
제왕: 빛과 그림자
호랑신랑뎐
인섹터
랭커
헬크래프트
더블클릭
버림받은 왕녀의 은밀한 침실
히어로메이커
파운더
팔이피플
도깨비 부른다
꼬리잡기
사이다걸
꿈의 기업
결혼생활 그림일기
버그이터
아, 쫌 참으세요 영주님!
똑 닮은 딸
물어보는 사이
파견체
소녀의 세계
오빠집이 비어서
최후의 금빛아이
북부 공작님을 유혹하겠습니다
경비실에서 안내방송 드립니다
이별 후 사내 결혼
불청객
신군
또다시, 계약 부부
하루의 하루
매지컬 급식:암살법사
미니어처 생활백서
아마도
루루라라 우리네 인생
순정말고 순종
황제와의 하룻밤
수영만화일기
백호랑
악녀 18세 공략기
이제야 연애
다시쓰는 연애사
퇴근 후에 만나요
슈퍼스타 천대리
아슈타르테
디나운스
집사, 주세요!
우산 없는 애
원작은 완결난 지 한참 됐습니다만
모스크바의 여명
나만의 고막남친
레지나레나 - 용서받지 못한 그대에게
메리의 불타는 행복회로
굿바이 유교보이
루크 비셸 따라잡기
그림자 신부
지옥연애환담
입술이 예쁜 남자
오늘의 비너스
역주행!
너를 돌려차는 방법
오로지 오로라
세번째 로망스
흔들리는 세계로부터
달로 만든 아이
별을 쫓는 소년들
사막에 핀 달
모노마니아
결혼공략
행운을 부탁해!
사랑의 헌옷수거함
-------------------tue-------------------
82
김부장
멸망 이후의 세계
내가 키운 S급들
하북팽가 막내아들
천마는 평범하게 살 수 없다
중증외상센터 : 골든 아워
헬58
용사가 돌아왔다
놓지마 정신줄 시즌3
여신강림
대학원 탈출일지
1을 줄게
랜덤채팅의 그녀!
시체기사 군터
삼국지톡
악몽의 형

## 요일별 조회순

In [96]:
import requests
from bs4 import BeautifulSoup

week_list = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'dailyPlus']

# 요일별 조회순
hit_order_list_by_day = []
# mon_hit_order_list_by_day = []
# ...

for week_name in week_list:
    globals()['{}_hit_order_list_by_day'.format(week_name)] = []
    print(globals()['{}_hit_order_list_by_day'.format(week_name)])
    hit_order_list_by_day.append(['{}_hit_order_list_by_day'.format(week_name)])
    
print(hit_order_list_by_day[1])
print(type(hit_order_list_by_day[1]))


for k in range(0, len(week_list)):
    print('-------------------' + week_list[k] + '-------------------')
    url = 'https://m.comic.naver.com/webtoon/weekday?sort=HIT&week=' + week_list[k]
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else : 
        print(response.status_code)

    # <span class="title_text">참교육</span>

    webtoon_title = soup.find_all("span", {"class":"title_text"})
    print(len(webtoon_title))
    for i in range(0, len(webtoon_title)):
        print(webtoon_title[i].text)
        hit_order_list_by_day[k].append(webtoon_title[i].text)

[]
[]
[]
[]
[]
[]
[]
[]
['tue_hit_order_list_by_day']
<class 'list'>
-------------------mon-------------------
81
소녀의 세계
뷰티풀 군바리
윈드브레이커
참교육
버림받은 왕녀의 은밀한 침실
퀘스트지상주의
팔이피플
장씨세가 호위무사
물어보는 사이
호랑신랑뎐
앵무살수
순정말고 순종
황제와의 하룻밤
오빠집이 비어서
똑 닮은 딸
신화급 귀속 아이템을 손에 넣었다
북부 공작님을 유혹하겠습니다
이별 후 사내 결혼
또다시, 계약 부부
리턴 투 플레이어
이제야 연애
퍼니게임
결혼생활 그림일기
절대검감
잔불의 기사
사이다걸
꼬리잡기
히어로메이커
아슈타르테
아, 쫌 참으세요 영주님!
퇴근 후에 만나요
원작은 완결난 지 한참 됐습니다만
미니어처 생활백서
입술이 예쁜 남자
다시쓰는 연애사
인섹터
더블클릭
집사, 주세요!
굿바이 유교보이
제왕: 빛과 그림자
오늘의 비너스
루루라라 우리네 인생
나만의 고막남친
그림자 신부
꿈의 기업
루크 비셸 따라잡기
헬크래프트
사랑의 헌옷수거함
레지나레나 - 용서받지 못한 그대에게
우산 없는 애
악녀 18세 공략기
랭커
파운더
불청객
메리의 불타는 행복회로
지옥연애환담
흔들리는 세계로부터
수영만화일기
모스크바의 여명
세번째 로망스
백호랑
하루의 하루
최후의 금빛아이
모노마니아
도깨비 부른다
버그이터
역주행!
행운을 부탁해!
오로지 오로라
별을 쫓는 소년들
경비실에서 안내방송 드립니다
달로 만든 아이
결혼공략
신군
사막에 핀 달
슈퍼스타 천대리
매지컬 급식:암살법사
파견체
아마도
디나운스
너를 돌려차는 방법
-------------------tue-------------------
82
여신강림
1을 줄게
김부장
하루만 네가 되고 싶어
대학원 탈출일지
마루는 강쥐
은주의 방 2~3부
내가 키운 S급들
중증외상센터 : 골든 아워
윌유메리미
올가미
랜덤채팅의 그녀!
멸망 이후의 세계
삼국지톡
원주민 공포만화
시한부인 줄 알았어요!
집이 없어
견우와 선녀
하북팽가 막내

## 요일별 인기순 DataFrame

In [97]:
import pandas as pd
popularity_order_list_by_day_df = pd.DataFrame([x for x in popularity_order_list_by_day])
popularity_order_list_by_day_df = popularity_order_list_by_day_df.T
popularity_order_list_by_day_df = popularity_order_list_by_day_df.rename(columns=popularity_order_list_by_day_df.iloc[0])
popularity_order_list_by_day_df = popularity_order_list_by_day_df.drop(popularity_order_list_by_day_df.index[0])
popularity_order_list_by_day_df.head()

,mon_popularity_order_by_day,tue_popularity_order_by_day,wed_popularity_order_by_day,thu_popularity_order_by_day,fri_popularity_order_by_day,sat_popularity_order_by_day,sun_popularity_order_by_day,dailyPlus_popularity_order_by_day
1,참교육,1을 줄게,내 남편과 결혼해줘,연애혁명,외모지상주의,99강화나무몽둥이,싸움독학,후궁의 초대
2,버림받은 왕녀의 은밀한 침실,여신강림,비즈니스 여친,재벌집 막내아들,어쩌다보니 천생연분,세레나,약한영웅,통 엣지 [완전판]
3,팔이피플,김부장,전지적 독자 시점,남편을 죽여줘요,나 혼자 만렙 뉴비,호랑이형님,입학용병,숲속에서 공작이 주운 것은
4,뷰티풀 군바리,하루만 네가 되고 싶어,먹는 인생,나노마신,개를 낳았다,취사병 전설이 되다,시월드가 내게 집착한다,일진담당일진
5,퀘스트지상주의,마루는 강쥐,내가 죽기로 결심한 것은,별을 삼킨 너에게,대학원 탈출일지,먹는 인생,사형소년,마녀의 소녀


## 요일별 여성_인기순 DataFrame

In [98]:
import pandas as pd
female_reader_popularity_order_list_by_day_df = pd.DataFrame([x for x in female_reader_popularity_order_list_by_day])
female_reader_popularity_order_list_by_day_df = female_reader_popularity_order_list_by_day_df.T
female_reader_popularity_order_list_by_day_df = female_reader_popularity_order_list_by_day_df.rename(columns=female_reader_popularity_order_list_by_day_df.iloc[0])
female_reader_popularity_order_list_by_day_df = female_reader_popularity_order_list_by_day_df.drop(female_reader_popularity_order_list_by_day_df.index[0])
female_reader_popularity_order_list_by_day_df.head()

,mon_female_reader_popularity_order_list_by_day,tue_female_reader_popularity_order_list_by_day,wed_female_reader_popularity_order_list_by_day,thu_female_reader_popularity_order_list_by_day,fri_female_reader_popularity_order_list_by_day,sat_female_reader_popularity_order_list_by_day,sun_female_reader_popularity_order_list_by_day,dailyPlus_female_reader_popularity_order_list_by_day
1,버림받은 왕녀의 은밀한 침실,1을 줄게,내 남편과 결혼해줘,남편을 죽여줘요,어쩌다보니 천생연분,세레나,시월드가 내게 집착한다,후궁의 초대
2,팔이피플,여신강림,비즈니스 여친,연애혁명,"언니, 이번 생엔 내가 왕비야",반드시 해피엔딩,자매전쟁,숲속에서 공작이 주운 것은
3,물어보는 사이,하루만 네가 되고 싶어,먹는 인생,재벌집 막내아들,개를 낳았다,아홉수 우리들,"곱게 키웠더니, 짐승",마녀의 소녀
4,호랑신랑뎐,마루는 강쥐,내가 죽기로 결심한 것은,별을 삼킨 너에게,세기말 풋사과 보습학원,먹는 인생,칼끝에 입술,신부가 필요해
5,똑 닮은 딸,대학원 탈출일지,마른 가지에 바람처럼,버려진 나의 최애를 위하여,대학원 탈출일지,작전명 순정,고백 취소도 되나?,선녀외전


## 요일별 남성_인기순 DataFrame

In [99]:
import pandas as pd
male_reader_popularity_order_list_by_day_df = pd.DataFrame([x for x in male_reader_popularity_order_list_by_day])
male_reader_popularity_order_list_by_day_df = male_reader_popularity_order_list_by_day_df.T
male_reader_popularity_order_list_by_day_df = male_reader_popularity_order_list_by_day_df.rename(columns=male_reader_popularity_order_list_by_day_df.iloc[0])
male_reader_popularity_order_list_by_day_df = male_reader_popularity_order_list_by_day_df.drop(male_reader_popularity_order_list_by_day_df.index[0])
male_reader_popularity_order_list_by_day_df.head()

,mon_male_reader_popularity_order_list_by_day,tue_male_reader_popularity_order_list_by_day,wed_male_reader_popularity_order_list_by_day,thu_male_reader_popularity_order_list_by_day,fri_male_reader_popularity_order_list_by_day,sat_male_reader_popularity_order_list_by_day,sun_male_reader_popularity_order_list_by_day,dailyPlus_male_reader_popularity_order_list_by_day
1,참교육,김부장,전지적 독자 시점,나노마신,외모지상주의,99강화나무몽둥이,싸움독학,통 엣지 [완전판]
2,퀘스트지상주의,멸망 이후의 세계,헬퍼 2 : 킬베로스,천마육성,나 혼자 만렙 뉴비,호랑이형님,입학용병,일진담당일진
3,뷰티풀 군바리,내가 키운 S급들,튜토리얼 탑의 고인물,재벌집 막내아들,광마회귀,취사병 전설이 되다,약한영웅,북경신보
4,장씨세가 호위무사,하북팽가 막내아들,무림서부,현실퀘스트,낙향문사전,프리드로우,사형소년,전생했더니 신입사원
5,윈드브레이커,천마는 평범하게 살 수 없다,나쁜사람,무사만리행,데드퀸,망나니 소교주로 환생했다,천하제일인,일신상의 이유로 잠시 휴재합니다


## 요일별 조회순 DataFrame

In [101]:
import pandas as pd
hit_order_list_by_day_df = pd.DataFrame([x for x in hit_order_list_by_day])
hit_order_list_by_day_df = hit_order_list_by_day_df.T
hit_order_list_by_day_df = hit_order_list_by_day_df.rename(columns=hit_order_list_by_day_df.iloc[0])
hit_order_list_by_day_df = hit_order_list_by_day_df.drop(hit_order_list_by_day_df.index[0])
hit_order_list_by_day_df.head()

,mon_hit_order_list_by_day,tue_hit_order_list_by_day,wed_hit_order_list_by_day,thu_hit_order_list_by_day,fri_hit_order_list_by_day,sat_hit_order_list_by_day,sun_hit_order_list_by_day,dailyPlus_hit_order_list_by_day
1,소녀의 세계,여신강림,내 남편과 결혼해줘,연애혁명,외모지상주의,호랑이형님,싸움독학,신부가 필요해
2,뷰티풀 군바리,1을 줄게,전지적 독자 시점,재벌집 막내아들,어쩌다보니 천생연분,프리드로우,입학용병,일진담당일진
3,윈드브레이커,김부장,연놈,나노마신,개를 낳았다,99강화나무몽둥이,"곱게 키웠더니, 짐승",후궁의 초대
4,참교육,하루만 네가 되고 싶어,먹는 인생,남편을 죽여줘요,대학원 탈출일지,세레나,약한영웅,숲속에서 공작이 주운 것은
5,버림받은 왕녀의 은밀한 침실,대학원 탈출일지,비즈니스 여친,별을 삼킨 너에게,나 혼자 만렙 뉴비,취사병 전설이 되다,시월드가 내게 집착한다,퇴마록


## 요일별 인기순, 여성_인기순, 남성_인기순, 조회순 CSV 파일 생성

In [102]:
by_day_df = pd.concat([popularity_order_list_by_day_df, 
                       female_reader_popularity_order_list_by_day_df, 
                       male_reader_popularity_order_list_by_day_df,
                       hit_order_list_by_day_df
                      ],axis=1)
by_day_df.head()

by_day_df.to_csv('네이버웹툰_요일별_인기순_여성_인기순_남성_인기순_조회순_202211181126.csv', encoding='utf-8-sig')

***

## 완결작 {장르} {정렬} 순

In [103]:
genre = ['PURE', 'ACTION', 'SPORTS', 'THRILL', 'FANTASY', 'DRAMA', 'DAILY', 'COMIC',
         'SENSIBILITY', 'HISTORICAL', 'EPISODE', 'OMNIBUS', 'STORY']
genre_lower = []
for i in genre:
    genre_lower.append(i.lower())
sort = ['ALL_READER', 'FEMALE_READER', 'MALE_READER', 'HIT']
order_list_name = ['popularity_', 'female_reader_popularity_', 'male_reader_popularity_', 'hit_']
print(genre_lower)

['pure', 'action', 'sports', 'thrill', 'fantasy', 'drama', 'daily', 'comic', 'sensibility', 'historical', 'episode', 'omnibus', 'story']


In [104]:
order_finished_work = []

for gr in genre_lower:
    for name in order_list_name:
        globals()['{}order_finished_work_{}'.format(name, gr)] = []
#         print('{}order_finished_work_{}'.format(name, genre))
        order_finished_work.append(['{}order_finished_work_{}'.format(name, gr)])

print(order_finished_work)
print(type(order_finished_work[0])) # [list]

[['popularity_order_finished_work_pure'], ['female_reader_popularity_order_finished_work_pure'], ['male_reader_popularity_order_finished_work_pure'], ['hit_order_finished_work_pure'], ['popularity_order_finished_work_action'], ['female_reader_popularity_order_finished_work_action'], ['male_reader_popularity_order_finished_work_action'], ['hit_order_finished_work_action'], ['popularity_order_finished_work_sports'], ['female_reader_popularity_order_finished_work_sports'], ['male_reader_popularity_order_finished_work_sports'], ['hit_order_finished_work_sports'], ['popularity_order_finished_work_thrill'], ['female_reader_popularity_order_finished_work_thrill'], ['male_reader_popularity_order_finished_work_thrill'], ['hit_order_finished_work_thrill'], ['popularity_order_finished_work_fantasy'], ['female_reader_popularity_order_finished_work_fantasy'], ['male_reader_popularity_order_finished_work_fantasy'], ['hit_order_finished_work_fantasy'], ['popularity_order_finished_work_drama'], ['fema

## 완결작 전체

In [9]:
import requests
from bs4 import BeautifulSoup

# {order_list_name}_popularity_order_finished_work_{genre_lower}
# 완결작 전체 인기순
popularity_order_finished_work_all = []
female_reader_popularity_order_finished_work_all = []
male_reader_popularity_order_finished_work_all = []
hit_order_finished_work_all = []

order_finished_work_all = [popularity_order_finished_work_all,
                          female_reader_popularity_order_finished_work_all,
                          male_reader_popularity_order_finished_work_all,
                          hit_order_finished_work_all]

# https://m.comic.naver.com/webtoon/finish?sort=ALL_READER

# https://m.comic.naver.com/webtoon/finish?sort=FEMALE_READER

# https://m.comic.naver.com/webtoon/finish?sort=MALE_READER

# https://m.comic.naver.com/webtoon/finish?sort=HIT
    
for k in range(0, len(order_finished_work_all)):
    print('-------------------' + sort[k] + '-------------------')
    url = 'https://m.comic.naver.com/webtoon/finish?sort=' + sort[k]
    print(url)
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else : 
        print(response.status_code)

    webtoon_title = soup.find_all("span", {"class":"title_text"})
    print(len(webtoon_title))
    for i in range(0, len(webtoon_title)):
        print(webtoon_title[i].text)
        order_finished_work_all[k].append(webtoon_title[i].text)


print(order_finished_work_all)

-------------------ALL_READER-------------------
https://m.comic.naver.com/webtoon/finish?sort=ALL_READER
30
2022 몰래보는 로맨스
이번 생도 잘 부탁해
갓 오브 하이스쿨
동생친구
화산귀환
독립일기
나의 플랏메이트
백수세끼
조조코믹스
생존로그
신의 탑
재혼 황후
죄송한데 주인공이세요?
오빠세끼
놓지마 정신줄
엔딩 후 서브남을 주웠다
100
온에어
무용과 남학생
별이삼샵
그들이 사귀는 세상
놓지마 정신줄 시즌2
여우놀이
구남친이 내게 반했다
기묘한 만화
이말년씨리즈
거래
초인의 시대
투신전생기
미혼남녀의 효율적 만남
-------------------FEMALE_READER-------------------
https://m.comic.naver.com/webtoon/finish?sort=FEMALE_READER
30
2022 몰래보는 로맨스
이번 생도 잘 부탁해
동생친구
나의 플랏메이트
독립일기
백수세끼
조조코믹스
재혼 황후
오빠세끼
화산귀환
엔딩 후 서브남을 주웠다
갓 오브 하이스쿨
그들이 사귀는 세상
무용과 남학생
온에어
놓지마 정신줄
여우놀이
죄송한데 주인공이세요?
구남친이 내게 반했다
생존로그
그 기사가 레이디로 사는 법
미혼남녀의 효율적 만남
불편한 관계
놓지마 정신줄 시즌2
독신마법사 기숙아파트
정년이
별이삼샵
희란국연가
남주서치
나를 길들여 봐, 차비서
-------------------MALE_READER-------------------
https://m.comic.naver.com/webtoon/finish?sort=MALE_READER
30
갓 오브 하이스쿨
화산귀환
2022 몰래보는 로맨스
이번 생도 잘 부탁해
신의 탑
생존로그
100
백수세끼
죄송한데 주인공이세요?
독립일기
별이삼샵
초인의 시대
투신전생기
이말년씨리즈
놓지마 정신줄
캐슬
동생친구
사신소년
놓지마 정신줄 시즌2
침묵의 밤
조조코믹스
거래
리트리츠
한림체육관
기묘한 만화
죽지 않으려면
가

In [10]:
import pandas as pd

In [11]:
order_finished_work_all_df = pd.DataFrame([x for x in order_finished_work_all])
order_finished_work_all_df = order_finished_work_all_df.T
order_finished_work_all_df.columns = ['popularity_order_finished_work_all',
                          'female_reader_popularity_order_finished_work_all',
                          'male_reader_popularity_order_finished_work_all',
                          'hit_order_finished_work_all']
# order_finished_work_all_df = order_finished_work_all_df.drop(order_finished_work_all_df.index[0])
order_finished_work_all_df

,popularity_order_finished_work_all,female_reader_popularity_order_finished_work_all,male_reader_popularity_order_finished_work_all,hit_order_finished_work_all
0,2022 몰래보는 로맨스,2022 몰래보는 로맨스,갓 오브 하이스쿨,[드라마원작] 유미의 세포들
1,이번 생도 잘 부탁해,이번 생도 잘 부탁해,화산귀환,스피릿 핑거스
2,갓 오브 하이스쿨,동생친구,2022 몰래보는 로맨스,갓 오브 하이스쿨
3,동생친구,나의 플랏메이트,이번 생도 잘 부탁해,신의 탑
4,화산귀환,독립일기,신의 탑,하이브 1~2
5,독립일기,백수세끼,생존로그,기기괴괴
6,나의 플랏메이트,조조코믹스,100,오!주예수여
7,백수세끼,재혼 황후,백수세끼,악의는 없다
8,조조코믹스,오빠세끼,죄송한데 주인공이세요?,공복의 저녁식사
9,생존로그,화산귀환,독립일기,복학왕


## 완결작 장르별 인기순, 여성_인기순, 남성_인기순, 조회순

In [21]:
genre = ['PURE', 'ACTION', 'SPORTS', 'THRILL', 'FANTASY', 'DRAMA', 'DAILY', 'COMIC',
         'SENSIBILITY', 'HISTORICAL', 'EPISODE', 'OMNIBUS', 'STORY']
genre_lower = []
for i in genre:
    genre_lower.append(i.lower())
sort = ['ALL_READER', 'FEMALE_READER', 'MALE_READER', 'HIT']
order_list_name = ['popularity_', 'female_reader_popularity_', 'male_reader_popularity_', 'hit_']
print(genre_lower)

['pure', 'action', 'sports', 'thrill', 'fantasy', 'drama', 'daily', 'comic', 'sensibility', 'historical', 'episode', 'omnibus', 'story']


In [22]:
order_finished_work = []

for gr in genre_lower:
    for name in order_list_name:
        globals()['{}order_finished_work_{}'.format(name, gr)] = []
#         print('{}order_finished_work_{}'.format(name, genre))
        order_finished_work.append(['{}order_finished_work_{}'.format(name, gr)])

print(order_finished_work)
print(type(order_finished_work[0])) # [list]

[['popularity_order_finished_work_pure'], ['female_reader_popularity_order_finished_work_pure'], ['male_reader_popularity_order_finished_work_pure'], ['hit_order_finished_work_pure'], ['popularity_order_finished_work_action'], ['female_reader_popularity_order_finished_work_action'], ['male_reader_popularity_order_finished_work_action'], ['hit_order_finished_work_action'], ['popularity_order_finished_work_sports'], ['female_reader_popularity_order_finished_work_sports'], ['male_reader_popularity_order_finished_work_sports'], ['hit_order_finished_work_sports'], ['popularity_order_finished_work_thrill'], ['female_reader_popularity_order_finished_work_thrill'], ['male_reader_popularity_order_finished_work_thrill'], ['hit_order_finished_work_thrill'], ['popularity_order_finished_work_fantasy'], ['female_reader_popularity_order_finished_work_fantasy'], ['male_reader_popularity_order_finished_work_fantasy'], ['hit_order_finished_work_fantasy'], ['popularity_order_finished_work_drama'], ['fema

In [23]:
order_finished_work_all_df = pd.DataFrame([x for x in order_finished_work_all])
order_finished_work_all_df = order_finished_work_all_df.T
order_finished_work_all_df.columns = ['popularity_order_finished_work_all',
                          'female_reader_popularity_order_finished_work_all',
                          'male_reader_popularity_order_finished_work_all',
                          'hit_order_finished_work_all']
# order_finished_work_all_df = order_finished_work_all_df.drop(order_finished_work_all_df.index[0])
order_finished_work_all_df

,popularity_order_finished_work_all,female_reader_popularity_order_finished_work_all,male_reader_popularity_order_finished_work_all,hit_order_finished_work_all
0,2022 몰래보는 로맨스,2022 몰래보는 로맨스,갓 오브 하이스쿨,[드라마원작] 유미의 세포들
1,이번 생도 잘 부탁해,이번 생도 잘 부탁해,화산귀환,스피릿 핑거스
2,갓 오브 하이스쿨,동생친구,2022 몰래보는 로맨스,갓 오브 하이스쿨
3,동생친구,나의 플랏메이트,이번 생도 잘 부탁해,신의 탑
4,화산귀환,독립일기,신의 탑,하이브 1~2
5,독립일기,백수세끼,생존로그,기기괴괴
6,나의 플랏메이트,조조코믹스,100,오!주예수여
7,백수세끼,재혼 황후,백수세끼,악의는 없다
8,조조코믹스,오빠세끼,죄송한데 주인공이세요?,공복의 저녁식사
9,생존로그,화산귀환,독립일기,복학왕


In [24]:
def divide_by_genre(aaa):
    num = genre.index(aaa) * 4
    return order_finished_work[num : num + 4]

divide_by_genre('COMIC'), divide_by_genre('FANTASY')

([['popularity_order_finished_work_comic'],
  ['female_reader_popularity_order_finished_work_comic'],
  ['male_reader_popularity_order_finished_work_comic'],
  ['hit_order_finished_work_comic']],
 [['popularity_order_finished_work_fantasy'],
  ['female_reader_popularity_order_finished_work_fantasy'],
  ['male_reader_popularity_order_finished_work_fantasy'],
  ['hit_order_finished_work_fantasy']])

In [25]:
for k in range(0, len(genre)):
    print('-------------------' + genre[k] + '-------------------')
    for j in range(0, len(sort)):
        url = 'https://m.comic.naver.com/webtoon/finish?sort=' + sort[j] + '&genre=' + genre[k]
        response = requests.get(url)

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
        else : 
            print(response.status_code)
            
        webtoon_title = soup.find_all("span", {"class":"title_text"})
        print(len(webtoon_title), sort[j])

-------------------PURE-------------------
30 ALL_READER
30 FEMALE_READER
30 MALE_READER
30 HIT
-------------------ACTION-------------------
30 ALL_READER
30 FEMALE_READER
30 MALE_READER
30 HIT
-------------------SPORTS-------------------
9 ALL_READER
9 FEMALE_READER
9 MALE_READER
9 HIT
-------------------THRILL-------------------
30 ALL_READER
30 FEMALE_READER
30 MALE_READER
30 HIT
-------------------FANTASY-------------------
30 ALL_READER
30 FEMALE_READER
30 MALE_READER
30 HIT
-------------------DRAMA-------------------
30 ALL_READER
30 FEMALE_READER
30 MALE_READER
30 HIT
-------------------DAILY-------------------
30 ALL_READER
30 FEMALE_READER
30 MALE_READER
30 HIT
-------------------COMIC-------------------
30 ALL_READER
30 FEMALE_READER
30 MALE_READER
30 HIT
-------------------SENSIBILITY-------------------
29 ALL_READER
29 FEMALE_READER
29 MALE_READER
29 HIT
-------------------HISTORICAL-------------------
19 ALL_READER
19 FEMALE_READER
19 MALE_READER
19 HIT
-------------------

In [26]:
for k in range(0, len(genre)):
    print('-------------------' + genre[k] + '-------------------')
    for j in range(0, len(sort)):
        url = 'https://m.comic.naver.com/webtoon/finish?sort=' + sort[j] + '&genre=' + genre[k]
        response = requests.get(url)

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
        else : 
            print(response.status_code)
            
        webtoon_title = soup.find_all("span", {"class":"title_text"})
        print('-------------------', len(webtoon_title), sort[j])
        
        for i in range(0, len(webtoon_title)):
            print(webtoon_title[i].text)
            divide_by_genre(genre[k])[j].append(webtoon_title[i].text)

-------------------PURE-------------------
------------------- 30 ALL_READER
2022 몰래보는 로맨스
이번 생도 잘 부탁해
동생친구
조조코믹스
재혼 황후
엔딩 후 서브남을 주웠다
그들이 사귀는 세상
구남친이 내게 반했다
미혼남녀의 효율적 만남
독신마법사 기숙아파트
불편한 관계
나를 길들여 봐, 차비서
희란국연가
남주서치
빨간맛 로맨스
해시의 신루
이두나!
좋은데 어떡해
2022 그걸 네가 왜!
힙한남자
대공님, 실수였어요!
바른연애 길잡이
화공, 해란
홍천기
짝사랑의 유서
당신의 여자가 되고 싶어요
흑막 여주가 날 새엄마로 만들려고 해
[드라마원작] 유미의 세포들
스피릿 핑거스
모락모락 왕세자님
------------------- 30 FEMALE_READER
2022 몰래보는 로맨스
이번 생도 잘 부탁해
동생친구
조조코믹스
재혼 황후
엔딩 후 서브남을 주웠다
그들이 사귀는 세상
구남친이 내게 반했다
미혼남녀의 효율적 만남
불편한 관계
독신마법사 기숙아파트
희란국연가
남주서치
나를 길들여 봐, 차비서
빨간맛 로맨스
해시의 신루
좋은데 어떡해
2022 그걸 네가 왜!
대공님, 실수였어요!
힙한남자
화공, 해란
바른연애 길잡이
이두나!
홍천기
당신의 여자가 되고 싶어요
짝사랑의 유서
흑막 여주가 날 새엄마로 만들려고 해
모락모락 왕세자님
스피릿 핑거스
2D남친 별책부록
------------------- 30 MALE_READER
2022 몰래보는 로맨스
이번 생도 잘 부탁해
동생친구
조조코믹스
독신마법사 기숙아파트
그들이 사귀는 세상
재혼 황후
엔딩 후 서브남을 주웠다
이두나!
찌질하지만 로맨스는 하고 싶어
미혼남녀의 효율적 만남
구남친이 내게 반했다
불편한 관계
나를 길들여 봐, 차비서
해시의 신루
힙한남자
빨간맛 로맨스
남주서치
바른연애 길잡이
[드라마원작] 유미의 세포들
2022 그걸 네가 왜!
희란국연가
수상한 문과장
대공님, 실수였어요!
[드라마원작] 알고있지만
오늘도 사랑스럽개
10년째 차이

------------------- 29 ALL_READER
밥 먹고 갈래요?
연우의 순정
다함께 이겨내요
환생동물학교
플랫다이어리
안녕 나의 모모로
모퉁이 뜨개방
일편단심화
우리집에 곰이 이사왔다
오늘도 핸드메이드!
마음의 숙제
다시 또 봄
나의 첫번째 새벽
겨울동화
미스테리오소
움비처럼
드로잉 레시피
홍차리브레
진눈깨비 소년
1인용 기분
소녀 연대기
여고생 임연수
보에
길에서 만나다
그린스마일
우리는 요정!
안녕, 나는 너를 좋아해
멍순이
징글정글
------------------- 29 FEMALE_READER
밥 먹고 갈래요?
연우의 순정
다함께 이겨내요
환생동물학교
플랫다이어리
모퉁이 뜨개방
안녕 나의 모모로
우리집에 곰이 이사왔다
오늘도 핸드메이드!
마음의 숙제
다시 또 봄
일편단심화
나의 첫번째 새벽
미스테리오소
홍차리브레
겨울동화
드로잉 레시피
진눈깨비 소년
1인용 기분
소녀 연대기
움비처럼
여고생 임연수
보에
안녕, 나는 너를 좋아해
길에서 만나다
그린스마일
멍순이
우리는 요정!
징글정글
------------------- 29 MALE_READER
밥 먹고 갈래요?
연우의 순정
다함께 이겨내요
환생동물학교
일편단심화
플랫다이어리
안녕 나의 모모로
움비처럼
우리집에 곰이 이사왔다
겨울동화
모퉁이 뜨개방
여고생 임연수
마음의 숙제
다시 또 봄
오늘도 핸드메이드!
드로잉 레시피
그린스마일
소녀 연대기
우리는 요정!
길에서 만나다
1인용 기분
나의 첫번째 새벽
미스테리오소
진눈깨비 소년
보에
홍차리브레
징글정글
멍순이
안녕, 나는 너를 좋아해
------------------- 29 HIT
밥 먹고 갈래요?
환생동물학교
겨울동화
우리집에 곰이 이사왔다
오늘도 핸드메이드!
홍차리브레
소녀 연대기
1인용 기분
모퉁이 뜨개방
연우의 순정
그린스마일
다함께 이겨내요
마음의 숙제
플랫다이어리
진눈깨비 소년
일편단심화
우리는 요정!
안녕 나의 모모로
나의 첫번째 새벽
멍순이
드로잉 레시피
징글정글
다시 또 봄
움비처럼
길에서 만나다
미스테리오소
안녕, 나는 너

In [27]:
for k in range(0, len(genre)):
    for i in divide_by_genre(genre[k]):
        print(i)

['popularity_order_finished_work_pure', '2022 몰래보는 로맨스', '이번 생도 잘 부탁해', '동생친구', '조조코믹스', '재혼 황후', '엔딩 후 서브남을 주웠다', '그들이 사귀는 세상', '구남친이 내게 반했다', '미혼남녀의 효율적 만남', '독신마법사 기숙아파트', '불편한 관계', '나를 길들여 봐, 차비서', '희란국연가', '남주서치', '빨간맛 로맨스', '해시의 신루', '이두나!', '좋은데 어떡해', '2022 그걸 네가 왜!', '힙한남자', '대공님, 실수였어요!', '바른연애 길잡이', '화공, 해란', '홍천기', '짝사랑의 유서', '당신의 여자가 되고 싶어요', '흑막 여주가 날 새엄마로 만들려고 해', '[드라마원작] 유미의 세포들', '스피릿 핑거스', '모락모락 왕세자님']
['female_reader_popularity_order_finished_work_pure', '2022 몰래보는 로맨스', '이번 생도 잘 부탁해', '동생친구', '조조코믹스', '재혼 황후', '엔딩 후 서브남을 주웠다', '그들이 사귀는 세상', '구남친이 내게 반했다', '미혼남녀의 효율적 만남', '불편한 관계', '독신마법사 기숙아파트', '희란국연가', '남주서치', '나를 길들여 봐, 차비서', '빨간맛 로맨스', '해시의 신루', '좋은데 어떡해', '2022 그걸 네가 왜!', '대공님, 실수였어요!', '힙한남자', '화공, 해란', '바른연애 길잡이', '이두나!', '홍천기', '당신의 여자가 되고 싶어요', '짝사랑의 유서', '흑막 여주가 날 새엄마로 만들려고 해', '모락모락 왕세자님', '스피릿 핑거스', '2D남친 별책부록']
['male_reader_popularity_order_finished_work_pure', '2022 몰래보는 로맨스', '이번 생도 잘 부탁해', '동생친구', '조조코믹스', '독신마법사 기숙아파트', '그들이 사귀는 세상', '재혼 황후', '엔딩 후

In [28]:
for i in divide_by_genre(genre[0]):
    print(i)

print(len(genre))

['popularity_order_finished_work_pure', '2022 몰래보는 로맨스', '이번 생도 잘 부탁해', '동생친구', '조조코믹스', '재혼 황후', '엔딩 후 서브남을 주웠다', '그들이 사귀는 세상', '구남친이 내게 반했다', '미혼남녀의 효율적 만남', '독신마법사 기숙아파트', '불편한 관계', '나를 길들여 봐, 차비서', '희란국연가', '남주서치', '빨간맛 로맨스', '해시의 신루', '이두나!', '좋은데 어떡해', '2022 그걸 네가 왜!', '힙한남자', '대공님, 실수였어요!', '바른연애 길잡이', '화공, 해란', '홍천기', '짝사랑의 유서', '당신의 여자가 되고 싶어요', '흑막 여주가 날 새엄마로 만들려고 해', '[드라마원작] 유미의 세포들', '스피릿 핑거스', '모락모락 왕세자님']
['female_reader_popularity_order_finished_work_pure', '2022 몰래보는 로맨스', '이번 생도 잘 부탁해', '동생친구', '조조코믹스', '재혼 황후', '엔딩 후 서브남을 주웠다', '그들이 사귀는 세상', '구남친이 내게 반했다', '미혼남녀의 효율적 만남', '불편한 관계', '독신마법사 기숙아파트', '희란국연가', '남주서치', '나를 길들여 봐, 차비서', '빨간맛 로맨스', '해시의 신루', '좋은데 어떡해', '2022 그걸 네가 왜!', '대공님, 실수였어요!', '힙한남자', '화공, 해란', '바른연애 길잡이', '이두나!', '홍천기', '당신의 여자가 되고 싶어요', '짝사랑의 유서', '흑막 여주가 날 새엄마로 만들려고 해', '모락모락 왕세자님', '스피릿 핑거스', '2D남친 별책부록']
['male_reader_popularity_order_finished_work_pure', '2022 몰래보는 로맨스', '이번 생도 잘 부탁해', '동생친구', '조조코믹스', '독신마법사 기숙아파트', '그들이 사귀는 세상', '재혼 황후', '엔딩 후

In [29]:
order_finished_work

[['popularity_order_finished_work_pure',
  '2022 몰래보는 로맨스',
  '이번 생도 잘 부탁해',
  '동생친구',
  '조조코믹스',
  '재혼 황후',
  '엔딩 후 서브남을 주웠다',
  '그들이 사귀는 세상',
  '구남친이 내게 반했다',
  '미혼남녀의 효율적 만남',
  '독신마법사 기숙아파트',
  '불편한 관계',
  '나를 길들여 봐, 차비서',
  '희란국연가',
  '남주서치',
  '빨간맛 로맨스',
  '해시의 신루',
  '이두나!',
  '좋은데 어떡해',
  '2022 그걸 네가 왜!',
  '힙한남자',
  '대공님, 실수였어요!',
  '바른연애 길잡이',
  '화공, 해란',
  '홍천기',
  '짝사랑의 유서',
  '당신의 여자가 되고 싶어요',
  '흑막 여주가 날 새엄마로 만들려고 해',
  '[드라마원작] 유미의 세포들',
  '스피릿 핑거스',
  '모락모락 왕세자님'],
 ['female_reader_popularity_order_finished_work_pure',
  '2022 몰래보는 로맨스',
  '이번 생도 잘 부탁해',
  '동생친구',
  '조조코믹스',
  '재혼 황후',
  '엔딩 후 서브남을 주웠다',
  '그들이 사귀는 세상',
  '구남친이 내게 반했다',
  '미혼남녀의 효율적 만남',
  '불편한 관계',
  '독신마법사 기숙아파트',
  '희란국연가',
  '남주서치',
  '나를 길들여 봐, 차비서',
  '빨간맛 로맨스',
  '해시의 신루',
  '좋은데 어떡해',
  '2022 그걸 네가 왜!',
  '대공님, 실수였어요!',
  '힙한남자',
  '화공, 해란',
  '바른연애 길잡이',
  '이두나!',
  '홍천기',
  '당신의 여자가 되고 싶어요',
  '짝사랑의 유서',
  '흑막 여주가 날 새엄마로 만들려고 해',
  '모락모락 왕세자님',
  '스피릿 핑거스',
  '2D남친 별책부록'],
 ['male_reader_popul

In [30]:
order_finished_work_df = pd.DataFrame([x for x in order_finished_work])
order_finished_work_df = order_finished_work_df.T
order_finished_work_df = order_finished_work_df.rename(columns=order_finished_work_df.iloc[0])
order_finished_work_df = order_finished_work_df.drop(order_finished_work_df.index[0])
order_finished_work_df.head()

,popularity_order_finished_work_pure,female_reader_popularity_order_finished_work_pure,male_reader_popularity_order_finished_work_pure,hit_order_finished_work_pure,popularity_order_finished_work_action,female_reader_popularity_order_finished_work_action,male_reader_popularity_order_finished_work_action,hit_order_finished_work_action,popularity_order_finished_work_sports,female_reader_popularity_order_finished_work_sports,...,male_reader_popularity_order_finished_work_episode,hit_order_finished_work_episode,popularity_order_finished_work_omnibus,female_reader_popularity_order_finished_work_omnibus,male_reader_popularity_order_finished_work_omnibus,hit_order_finished_work_omnibus,popularity_order_finished_work_story,female_reader_popularity_order_finished_work_story,male_reader_popularity_order_finished_work_story,hit_order_finished_work_story
1,2022 몰래보는 로맨스,2022 몰래보는 로맨스,2022 몰래보는 로맨스,[드라마원작] 유미의 세포들,생존로그,생존로그,생존로그,소녀더와일즈,더 복서,더 복서,...,2022 몰래보는 로맨스,[드라마원작] 유미의 세포들,조조코믹스,조조코믹스,이말년씨리즈,기기괴괴,이번 생도 잘 부탁해,이번 생도 잘 부탁해,갓 오브 하이스쿨,스피릿 핑거스
2,이번 생도 잘 부탁해,이번 생도 잘 부탁해,이번 생도 잘 부탁해,스피릿 핑거스,100,싸이코 리벤지,100,인생존망,조석축구만화,조석축구만화,...,독립일기,밥 먹고 갈래요?,기묘한 만화,기묘한 만화,조조코믹스,2015 소름,갓 오브 하이스쿨,동생친구,화산귀환,갓 오브 하이스쿨
3,동생친구,동생친구,동생친구,오!주예수여,초인의 시대,100,초인의 시대,전설의 레전드,라이징패스트볼,그린보이,...,놓지마 정신줄,슈퍼 시크릿,이말년씨리즈,즐거우리 우리네 인생,기묘한 만화,2011 미스테리 단편,동생친구,나의 플랏메이트,이번 생도 잘 부탁해,신의 탑
4,조조코믹스,조조코믹스,조조코믹스,[드라마원작] 내 ID는 강남미인!,투신전생기,사신소년,투신전생기,부활남,그린보이,파도를 찾아라!,...,놓지마 정신줄 시즌2,내 ID는 강남미인! - 둘만의 시간,즐거우리 우리네 인생,미물,2020 호랑 공포 단편선,2015 사이,화산귀환,백수세끼,신의 탑,하이브 1~2
5,재혼 황후,재혼 황후,독신마법사 기숙아파트,슈퍼 시크릿,캐슬,정글쥬스,캐슬,폭풍의 전학생,파도를 찾아라!,라이징패스트볼,...,한림체육관,마음의소리,2020 호랑 공포 단편선,2020 호랑 공포 단편선,미물,2017 사이다를 부탁해!,나의 플랏메이트,재혼 황후,생존로그,오!주예수여


In [ ]:
order_finished_work_df.to_csv('네이버웹툰_완결작_인기순_여성_인기순_남성_인기순_조회순_202211181457.csv', encoding='utf-8-sig')

---

## 요일별 웹툰 랭크

In [ ]:
day_webtoon_df = pd.read_csv('네이버웹툰_요일별_인기순_여성_인기순_남성_인기순_조회순_202211181126.csv')
day_webtoon_df

In [ ]:
day_webtoon_df_2 = day_webtoon_df.rename(columns={'Unnamed: 0':'rank'})
day_webtoon_df_2

In [ ]:
exemple_df = day_webtoon_df_2[['rank', 'mon_popularity_order_by_day']]
exemple_df = exemple_df.dropna(axis=0)
exemple_df

In [ ]:
dictionary = {}
for i in range(0, len(exemple_df['mon_popularity_order_by_day'])):
    dictionary[i] = exemple_df['mon_popularity_order_by_day'][i]
# print(dictionary)

exemple_df.rename(index=dictionary, inplace=True)

In [ ]:
exemple_df_2 = day_webtoon_df_2[['rank', 'tue_popularity_order_by_day']]
exemple_df_2 = exemple_df_2.dropna(axis=0)
exemple_df_2

dictionary = {}
for i in range(0, len(exemple_df_2[exemple_df_2.columns[1]])):
    dictionary[i] = exemple_df_2[exemple_df_2.columns[1]][i]
print(dictionary)

In [ ]:
exemple_df = exemple_df.rename(columns={exemple_df.columns[1] : 'title', 'rank':'mon_popularity_order_by_day'})
exemple_df.head()

In [ ]:
exemple_df_2 = exemple_df_2.rename(columns={exemple_df_2.columns[1] : 'title', 'rank':'tue_popularity_order_by_day'})
exemple_df_2.head()

In [ ]:
merge_outer = pd.merge(exemple_df, exemple_df_2, how='outer',on='title')
merge_outer

In [ ]:
for e in range(3, len(day_webtoon_df_2.columns)):
    exemple_df_3 = day_webtoon_df_2[['rank', day_webtoon_df_2.columns[e]]]
    exemple_df_3 = exemple_df_3.dropna(axis=0)

    dictionary = {}
    for i in range(0, len(exemple_df_3[exemple_df_3.columns[1]])):
        dictionary[i] = exemple_df_3[exemple_df_3.columns[1]][i]

    exemple_df_3 = exemple_df_3.rename(columns={'rank':exemple_df_3.columns[1], exemple_df_3.columns[1] : 'title'})

    merge_outer = pd.merge(merge_outer, exemple_df_3, how='outer',on='title')

merge_outer

In [ ]:
day_webtoon_rank_nan = merge_outer
day_webtoon_rank_nan

In [ ]:
day_webtoon_rank = day_webtoon_rank_nan.fillna(0)
day_webtoon_rank

In [ ]:
day_webtoon_rank.to_csv('요일별_웹툰의_인기순_여성_인기순_남성_인기순_조회순_순위값.csv', encoding='utf-8-sig', index=False)

---

## 완결작 웹툰 랭크

In [ ]:
finished_webtoon_df = pd.read_csv('네이버웹툰_완결작_인기순_여성_인기순_남성_인기순_조회순_202211181457.csv')
finished_webtoon_df

In [ ]:
finished_webtoon_df = finished_webtoon_df.rename(columns={'Unnamed: 0':'rank'})
finished_webtoon_df
print(len(finished_webtoon_df.columns))

In [ ]:
example_df = finished_webtoon_df[['rank', finished_webtoon_df.columns[1]]]
example_df = example_df.dropna(axis=0)
example_df

In [ ]:
dictionary = {}
for i in range(0, len(example_df[example_df.columns[1]])):
    dictionary[i] = example_df[example_df.columns[1]][i]

example_df = example_df.rename(columns={'rank':example_df.columns[1], example_df.columns[1] : 'title'})
example_df


In [ ]:
example_df_2 = finished_webtoon_df[['rank', finished_webtoon_df.columns[2]]]
example_df_2 = example_df_2.dropna(axis=0)
example_df_2

In [ ]:
dictionary = {}
for i in range(0, len(example_df_2[example_df_2.columns[1]])):
    dictionary[i] = example_df_2[example_df_2.columns[1]][i]

example_df_2 = example_df_2.rename(columns={'rank':example_df_2.columns[1], example_df_2.columns[1] : 'title'})
example_df_2


In [ ]:
merge_outer = pd.merge(example_df, example_df_2, how='outer',on='title')
merge_outer

In [ ]:
for e in range(3, len(finished_webtoon_df.columns)):
    example_df = finished_webtoon_df[['rank', finished_webtoon_df.columns[e]]]
    example_df = example_df.dropna(axis=0)

    dictionary = {}
    for i in range(0, len(example_df[example_df.columns[1]])):
        dictionary[i] = example_df[example_df.columns[1]][i]

    example_df = example_df.rename(columns={'rank':example_df.columns[1], example_df.columns[1] : 'title'})

    merge_outer = pd.merge(merge_outer, example_df, how='outer',on='title')

merge_outer

In [ ]:
finished_webtoon_rank_nan = merge_outer
finished_webtoon_rank_nan

In [ ]:
finished_webtoon_rank = finished_webtoon_rank_nan.fillna(0)
finished_webtoon_rank

In [ ]:
finished_webtoon_rank.to_csv('완결작_장르별_웹툰의_인기순_여성_인기순_남성_인기순_조회순_순위값.csv', encoding='utf-8-sig', index=False)

---